In [3]:
using JuMP, Ipopt
using Plots
gr()

Plots.GRBackend()

### 1. MPC with JuMP, using code from [MIT's Juliacon presentation](https://github.com/tkoolen/Robocon2017Tutorial.jl).

In [22]:
# Set up model of particle 

model = Model(solver=IpoptSolver())

# Constants
Δt = 0.1
num_time_steps = 20
max_acceleration = 0.5

# Decision variables
@variables model begin
    position[1:2, 1:num_time_steps]
    velocity[1:2, 1:num_time_steps]
    -max_acceleration <= acceleration[1:2, 1:num_time_steps] <= max_acceleration
end

# Dynamics constraints
@constraint(model, [i=2:num_time_steps, j=1:2],
            velocity[j, i] == velocity[j, i - 1] + acceleration[j, i - 1] * Δt)
@constraint(model, [i=2:num_time_steps, j=1:2],
            position[j, i] == position[j, i - 1] + velocity[j, i - 1] * Δt)

# Cost function: minimize final position and final velocity
@objective(model, Min, 
    100 * sum(position[:, end].^2) + sum(velocity[:, end].^2))

# Initial conditions:
@constraint(model, position[:, 1] .== [1, 0])
@constraint(model, velocity[:, 1] .== [0, 1])

solve(model)

This is Ipopt version 3.12.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      232
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:      120
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       40
                     variables with only upper bounds:        0
Total number of equality constraints.................:       80
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

:Optimal

In [23]:
# Extract the solution from the model
q = getvalue(position)
v = getvalue(velocity)
u = getvalue(acceleration)

2×20 Array{Float64,2}:
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5  …  -0.5  -0.5  -0.5  -0.5   0.5  0.0
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5     -0.5  -0.5  -0.5  -0.5  -0.5  0.0

In [24]:
using Plots
# Use the GR backend for Plots.jl, because it's fast
gr()

Plots.GRBackend()

In [25]:
# The @gif macro creates an animate plot, which lets us draw the
# optimized trajectory of the brick as a function of time
anim = @animate for i = 1:num_time_steps
    plot(q[1, :], q[2, :], xlim=(-1.1, 1.1), ylim=(-1.1, 1.1))
    plot!([q[1, i]], [q[2, i]], marker=(:hex, 6))
end
gif(anim, "img/mpc1.gif", fps = 30)

INFO: Saved animation to /Users/Kazu/Code/julia/JuliaExperiments/img/mpc1.gif


Plots.AnimatedGif("/Users/Kazu/Code/julia/JuliaExperiments/img/mpc1.gif")

To do MPC, we can optimize a trajectory at each timestep, execute the first command, then replan at the next timestep.

TODO: 

* Try adding noise

### 2. Local QP controller

In [4]:
# Constants
Δt = 0.1
num_time_steps = 20
max_acceleration = 0.5
kp = 1.5
kd = 2*sqrt(1.)

function qp_controller(pos_cur, vel_cur, pos_des, vel_des)
    model = Model(solver=IpoptSolver(print_level=0))
    
    # Decision variables
    @variables model begin
        -max_acceleration <= acceleration[1:2] <= max_acceleration
    end

    # Manual warm-start
#     if isdefined(:prev_solution)
#         setvalue(acceleration, prev_solution)
#     end
    
    # Don't really need dynamics constraint because the dynamics model is just F=m*a
    
    # Cost function: spring damper behavior (PD controller) on \ddot{p} between current and goal
    # TODO try converting this to Q, C matrices
    @objective(model, Min, sum((kp*(pos_des - pos_cur) + kd*(vel_des - vel_cur) - acceleration).^2))

    solve(model)
#     global prev_solution = getvalue(acceleration)
    return getvalue(acceleration)
end

qp_controller (generic function with 1 method)

In [5]:
pos_des = [-1,1]
vel_des = [0,0]
q = [1,0]
v = [0,0]

anim = @animate for i in 1:75
    # Plot the current and goal positions
    plot([pos_des[1]], [pos_des[2]], marker=(:circle, 5), xlim=(-2.1, 2.1), ylim=(-2.1, 2.1), legend=:none)
    plot!([q[1]], [q[2]], marker=(:circle, 10))
    
    # Run the MPC control optimization
#     tic()
    accel = qp_controller(q, v, pos_des, vel_des)
#     print(i, " ")
#     toc()
    
    # Apply the planned acceleration and simulate one step in time
    v += accel * Δt
    q += v * Δt
end

gif(anim, "img/qp_traj.gif")


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



INFO: Saved animation to /Users/Kazu/Code/julia/JuliaExperiments/img/qp_traj.gif


Plots.AnimatedGif("/Users/Kazu/Code/julia/JuliaExperiments/img/qp_traj.gif")